In [1]:
import os
import sys
PROJECT_PATH = "/home/albert/Baikal-ML/" #insert your project path
sys.path.append(f"{PROJECT_PATH}")

import numpy as np
import uproot as ur
import pandas as pd

from data.sql_manager.sql_engine import LocalSQLiteDatabase as DB
from data.root_manager.root_reader import RootReader

In [2]:
prty_type = 'muatm'
path_to_roots = f'{PROJECT_PATH}data/mock_MC_2020/{prty_type}/root/all/'

files = os.listdir(f'{path_to_roots}')[0:1]
paths = sorted([f"{path_to_roots}{file}" for file in files])
prefixes = [f"{prty_type}_{file.split('.')[0]}_" for file in files]

paths, prefixes

(['/home/albert/Baikal-ML/data/mock_MC_2020/muatm/root/all/66474.root'],
 ['muatm_66474_'])

In [3]:
for _, (path, prefix) in enumerate(zip(paths, prefixes)):
    with ur.open(path) as rf:
        rr = RootReader(rf, prefix=prefix)
        if _ == 0:
            df_ev = rr.read_events_as_df()
            df_puls = rr.read_pulses_as_df()
            df_muons = rr.read_ind_mu_as_df()
            df_OM_coords = rr.read_OM_coords()
        else:
            df_ev = pd.concat([df_ev, rr.read_events_as_df()])
            df_puls = pd.concat([df_puls, rr.read_pulses_as_df()])
            df_muons = pd.concat([df_muons, rr.read_ind_mu_as_df()])
            df_OM_coords = pd.concat([df_OM_coords, rr.read_OM_coords()])

In [4]:
df_puls.head(10)

,PulsesChID,PulsesAmpl,PulsesTime,PulsesFlg,ev_id,is_signal,mu_local_id,cluster_id,string_id
12,342,0.930025,-557.693542,0,muatm_66474_0,False,NaN,1,9
45,470,0.908726,-432.473419,0,muatm_66474_0,False,NaN,1,13
60,537,0.478152,-353.609772,0,muatm_66474_0,False,NaN,1,14
34,421,0.401339,-346.686493,0,muatm_66474_0,False,NaN,1,11
0,296,0.945107,-319.191589,0,muatm_66474_0,False,NaN,1,8
69,571,0.964510,-269.142456,0,muatm_66474_0,False,NaN,1,15
51,490,1.336054,-266.403076,0,muatm_66474_0,False,NaN,1,13
25,388,1.064100,-217.151398,0,muatm_66474_0,False,NaN,1,10
3,309,0.875028,-69.726913,0,muatm_66474_0,False,NaN,1,8
26,391,0.617015,-56.601349,0,muatm_66474_0,False,NaN,1,10


In [7]:
db = DB(db_name='test.db')
db.store_dataframe(df_puls, 'test')

Connected to SQLite database: test.db
DataFrame has been optimized!
Table test created successfully.
DataFrame stored successfully in table test.


In [8]:
retrieved_df = db.retrieve_dataframe('SELECT * FROM test where is_signal=1')
retrieved_df

DataFrame retrieved successfully.


,PulsesChID,PulsesAmpl,PulsesTime,PulsesFlg,ev_id,is_signal,mu_local_id,cluster_id,string_id
0,315,1.253025,1767.186523,-999999,muatm_66474_0,1,0.0,1,8
1,352,8.557582,1862.051025,1000002,muatm_66474_0,1,1.0,1,9
2,353,5.684482,1863.623657,1000002,muatm_66474_0,1,1.0,1,9
3,386,7.361794,2048.952637,-999998,muatm_66474_0,1,1.0,1,10
4,1223,2.864390,1659.194946,-999999,muatm_66474_1,1,0.0,4,33
...,...,...,...,...,...,...,...,...,...
139353,768,3.362602,2032.586426,-999976,muatm_66474_25699,1,23.0,2,21
139354,802,2.414667,2100.976318,-999988,muatm_66474_25699,1,11.0,2,22
139355,801,0.430441,2164.877930,1000032,muatm_66474_25699,1,31.0,2,22
139356,799,2.086887,2221.416260,-999997,muatm_66474_25699,1,2.0,2,22
